In [45]:
import pandas as pd
merged_data=pd.read_csv("food_recommendation_merged1.csv")
merged_data

,User_ID,Food_ID,Rating,Name,C_Type,Veg_Non,Describe,C_Type_encoded,Veg_Non_encoded
0,1,88,4.0,peri peri chicken satay,Snack,non-veg,"boneless skinless chicken thigh (trimmed), sal...",12,0
1,1,46,3.0,steam bunny chicken bao,Japanese,non-veg,"buns, all purpose white flour, dry yeast, suga...",8,0
2,1,24,5.0,green lentil dessert fudge,Dessert,veg,"whole moong beans, cow ghee, raisins, whole mi...",3,1
3,1,25,4.0,cashew nut cookies,Dessert,veg,"cashew paste, ghee, khaand (a sweetening agent...",3,1
4,2,49,1.0,christmas tree pizza,Italian,veg,"pizza dough (2 boules), red pepper, red onion,...",7,1
...,...,...,...,...,...,...,...,...,...
506,99,65,7.0,almond white chocolate gujiya,Japanese,veg,"all-purpose flour, clarified butter (ghee), wa...",8,1
507,99,22,1.0,baked namak para,Snack,veg,"whole wheat flour (atta), refined flour (maida...",12,1
508,100,24,10.0,green lentil dessert fudge,Dessert,veg,"whole moong beans, cow ghee, raisins, whole mi...",3,1
509,100,233,10.0,gluten free almond cake,Healthy Food,veg,"Almond Powder, Egg, Honey, Baking Soda, Vanill...",5,1


In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv('food_recommendation_merged1.csv') 

def content_based_recommendation(cuisine_type, veg_non, num_recommendations=5):

    filtered_data = data[(data['C_Type'] == cuisine_type) & (data['Veg_Non_encoded'] == veg_non)]

    if filtered_data.empty:
        print("No data found for the selected criteria.")
        return pd.DataFrame(columns=['Name', 'Cuisine', 'Rating'])
    
    # Handle duplicate food items (e.g., averaging the ratings for duplicates)
    filtered_data = filtered_data.groupby('Name').agg({'Rating': 'mean', 'C_Type': 'first'}).reset_index()

    tfidf = TfidfVectorizer(stop_words='english')
    filtered_data['Name'] = filtered_data['Name'].fillna('')  # Use 'Name' column instead of 'Describe'
    tfidf_matrix = tfidf.fit_transform(filtered_data['Name'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    indices = pd.Series(range(len(filtered_data)), index=filtered_data['Name']).drop_duplicates()

    try:
        idx = indices.sample(1).iloc[0] 
    except ValueError:
        print("Unable to select a sample from the filtered data.")
        return pd.DataFrame(columns=['Name', 'Cuisine', 'Rating'])

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:num_recommendations + 1] 

    food_indices = [i[0] for i in sim_scores if i[0] < len(filtered_data)]
    similarity_values = [i[1] for i in sim_scores]
    
    recommendations = filtered_data.iloc[food_indices][['Name', 'C_Type', 'Rating']]

    recommendations = recommendations.rename(columns={'C_Type': 'Cuisine'})
    
    recommendations['Cosine Similarity'] = similarity_values
    
    return recommendations




In [5]:
cuisine_type = input("Enter the cuisine type (e.g., 'Indian'): ")
veg_non_input = input("Enter 1 for Veg or 0 for Non-Veg: ")

veg_non = int(veg_non_input)

num_recommendations = 5

recommended_foods = content_based_recommendation(cuisine_type, veg_non, num_recommendations)

print("\nRecommended Foods with Cosine Similarity Values:")
recommended_foods

Enter the cuisine type (e.g., 'Indian'):  Indian
Enter 1 for Veg or 0 for Non-Veg:  0



Recommended Foods with Cosine Similarity Values:


,Name,Cuisine,Rating,Cosine Similarity
38,methi chicken masala,Indian,7.0,0.457698
17,chicken masala,Indian,5.0,0.091131
23,chicken tikka,Indian,8.0,0.078997
47,tandoori chicken,Indian,10.0,0.078997
13,chicken biryani,Indian,3.5,0.073843


In [6]:
def collaborative_recommendation(user_id, num_recommendations=5):
    data_unique = data.drop_duplicates(subset=['Food_ID', 'User_ID'], keep='first')

    pivot_table = data_unique.pivot(index='Food_ID', columns='User_ID', values='Rating').fillna(0)

    user_ratings = pivot_table.values.T  
    cosine_sim = cosine_similarity(user_ratings)

    if user_id not in pivot_table.columns:
        print(f"User ID {user_id} not found in the dataset.")
        return pd.DataFrame(columns=['Name', 'Cuisine', 'Rating'])
    
    user_index = pivot_table.columns.get_loc(user_id)

    user_similarities = cosine_sim[user_index]
    similar_users = user_similarities.argsort()[-num_recommendations-1:-1][::-1]

    print(f"Cosine similarity scores for User ID {user_id}:")
    for i in range(num_recommendations):
        similar_user_id = pivot_table.columns[similar_users[i]]
        similarity_score = user_similarities[similar_users[i]]
        print(f"User ID {similar_user_id} - Similarity: {similarity_score:.4f}")
    

    recommended_foods = set()
    for similar_user in similar_users:
        similar_user_ratings = pivot_table.iloc[:, similar_user]
        top_rated_foods = similar_user_ratings[similar_user_ratings > 0].index
        recommended_foods.update(top_rated_foods) 

    recommended_foods = list(recommended_foods)  
    recommendations = data[data['Food_ID'].isin(recommended_foods)]
    

    recommendations = (
        recommendations.groupby(['Food_ID', 'Name', 'C_Type'], as_index=False)
        .agg({'Rating': 'mean'})
        .rename(columns={'C_Type': 'Cuisine'})
    )
    

    recommendations = recommendations.sort_values(by='Rating', ascending=False)
    
    return recommendations

try:
    user_id = int(input("Enter User ID (e.g., 1): "))  # Accept user input for user_id
    recommendations = collaborative_recommendation(user_id, num_recommendations=5)
    if not recommendations.empty:
        print("\nRecommended Foods:")
        print(recommendations)
    else:
        print("No recommendations available.")
except ValueError:
    print("Invalid input! Please enter a valid user ID.")


Enter User ID (e.g., 1):  52


Cosine similarity scores for User ID 52:
User ID 10 - Similarity: 0.5600
User ID 74 - Similarity: 0.4557
User ID 99 - Similarity: 0.2473
User ID 49 - Similarity: 0.0965
User ID 19 - Similarity: 0.0040

Recommended Foods:
    Food_ID                                       Name       Cuisine  \
25      276                        apple and pear cake  Healthy Food   
12       77                    grilled lemon margarita      Beverage   
23      249                       vegetable manchurian       Chinese   
17      139      surmai curry with lobster butter rice          Thai   
19      171                                 corn pulao        Indian   
0         1                        summer squash salad  Healthy Food   
13       84                              filter coffee      Beverage   
6        44                   andhra pan fried pomfret        Indian   
14       93             buldak (hot and spicy chicken)      Japanese   
10       69               banana and maple ice lollies     

In [7]:
import pandas as pd


def hybrid_recommendation(user_id, cuisine_type, veg_non, num_recommendations=5):
    content_recs = content_based_recommendation(cuisine_type, veg_non, num_recommendations)
    collaborative_recs = collaborative_recommendation(user_id, num_recommendations)
    

    hybrid_recs = pd.concat([content_recs, collaborative_recs]).drop_duplicates().head(num_recommendations)
    
    return hybrid_recs

user_id = int(input("Enter User ID: "))
cuisine_type = input("Enter Cuisine Type (e.g., Italian, Chinese, etc.): ")
veg_non = input("Is it Veg or Non-Veg? (Enter 'Veg' or 'Non-Veg'): ")

veg_non_encoded = 1 if veg_non.lower() == "veg" else 0

recommendations = hybrid_recommendation(user_id, cuisine_type, veg_non_encoded)

if recommendations.empty:
    print("No recommendations found.")
else:
    print("Top Recommendations:")
    print(recommendations[['Name', 'Cuisine', 'Rating']])


Enter User ID:  21
Enter Cuisine Type (e.g., Italian, Chinese, etc.):  Indian
Is it Veg or Non-Veg? (Enter 'Veg' or 'Non-Veg'):  


Cosine similarity scores for User ID 21:
User ID 38 - Similarity: 0.3729
User ID 6 - Similarity: 0.0925
User ID 31 - Similarity: 0.0768
User ID 55 - Similarity: 0.0632
User ID 90 - Similarity: 0.0538
Top Recommendations:
                Name Cuisine  Rating
17    chicken masala  Indian     5.0
23     chicken tikka  Indian     8.0
47  tandoori chicken  Indian    10.0
13   chicken biryani  Indian     3.5
25      dahi chicken  Indian     6.0
